In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta
import time


import warnings
warnings.filterwarnings('ignore')

In [12]:
def single_search(url, driver, Snapshot_date, checking_date_type, checkout_date_type):
    

    hotels = []
    index = 1
    
    driver.get(url)
    for i in [1,2]:
        button=driver.find_element(by =By.CSS_SELECTOR,value =f'[data-stid="show-more-results"]')
        button.click()
        time.sleep(2) 
    
    text_boxes_list = driver.find_elements(by= By.CLASS_NAME , value='uitk-card-content-section')
    
    for t_box in text_boxes_list:
        try:
            hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Grade":None,
                     "Num of Reviews":None,"Curr Price":None,"Original Price":None,"Distance from center":None,
                     "Type of room":None,"Location grade":None ,"Extra Data":None}
            
            name = t_box.find_element(by= By.CLASS_NAME , value="uitk-heading").text
            score = t_box.find_element(By.CLASS_NAME, 'uitk-spacing-padding-inlineend-half').text
            price = t_box.find_element(By.CSS_SELECTOR, "div[data-test-id='price-summary-message-line']").text.split('$')
          
            hotel['Snapshot'] = Snapshot_date
            hotel['Index'] = index
            hotel['Hotel Name'] = name
            hotel['TTT'] = (checking_date_type - Snapshot_date).days
            hotel['LOS'] = (checkout_date_type - checking_date_type).days
            hotel['Grade'] = score 
            '''hotel['Curr Price'] = price[-1]
            if len(price) > 3:
                hotel['Original Price'] = price[1].split('\n')[0]'''
            hotels.append(hotel)
            
            index = index + 1
        except:
             print('error')
    
    return pd.DataFrame.from_dict(hotels)

In [13]:
#Allows us to run on different computers without a problem

path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': r"C:\Users\yarin\chromedriver.exe", 'yair':'C:\Windows\chromdriver.exe' }
driver = webdriver.Chrome(path['yarin'])

In [14]:
# Get today's date
now = datetime.now()
Snapshot_date =now
# Get end date 35 days from now
end_date = now + timedelta(days=35)

# Create a list of all dates between today and end date
date_list = []
while now <= end_date:
    date_list.append(now.strftime('%Y-%m-%d'))
    now += timedelta(days=1)


#Creates a list whose every member is a list of checking and checkout time

combinations_list =[]
for i in range(1,len(date_list)-5):
    for j in range(1,6):
        combinations = [date_list[i],date_list[i+j]]
        combinations_list.append(combinations)

print(combinations_list , len(combinations_list)) 

[['2023-02-08', '2023-02-09'], ['2023-02-08', '2023-02-10'], ['2023-02-08', '2023-02-11'], ['2023-02-08', '2023-02-12'], ['2023-02-08', '2023-02-13'], ['2023-02-09', '2023-02-10'], ['2023-02-09', '2023-02-11'], ['2023-02-09', '2023-02-12'], ['2023-02-09', '2023-02-13'], ['2023-02-09', '2023-02-14'], ['2023-02-10', '2023-02-11'], ['2023-02-10', '2023-02-12'], ['2023-02-10', '2023-02-13'], ['2023-02-10', '2023-02-14'], ['2023-02-10', '2023-02-15'], ['2023-02-11', '2023-02-12'], ['2023-02-11', '2023-02-13'], ['2023-02-11', '2023-02-14'], ['2023-02-11', '2023-02-15'], ['2023-02-11', '2023-02-16'], ['2023-02-12', '2023-02-13'], ['2023-02-12', '2023-02-14'], ['2023-02-12', '2023-02-15'], ['2023-02-12', '2023-02-16'], ['2023-02-12', '2023-02-17'], ['2023-02-13', '2023-02-14'], ['2023-02-13', '2023-02-15'], ['2023-02-13', '2023-02-16'], ['2023-02-13', '2023-02-17'], ['2023-02-13', '2023-02-18'], ['2023-02-14', '2023-02-15'], ['2023-02-14', '2023-02-16'], ['2023-02-14', '2023-02-17'], ['2023-02

In [15]:
# for dev
combinations_list = [combinations_list[0]]
# END - for dev



city = "New%20York%20%28and%20vicinity%29%2C%20New%20York%2C%20United%20States%20of%20America"
city_latlong = 'latLong=40.75668%2C-73.98647'
city_region_id = '178293'
numOfAdults=2
numOfChild=0
numOfRooms=1
expedia_df = pd.DataFrame(columns=['Snapshot','Index','Hotel Name','TTT' ,'LOS' ,"Grade","Num of Reviews","Curr Price","Original Price",
                                   "Distance from center","Type of room","Location grade", "Extra Data"])
for comb in combinations_list:
    checking_date_type = datetime.strptime(comb[0], '%Y-%m-%d')
    checkout_date_type = datetime.strptime(comb[1], '%Y-%m-%d')
    checking_date = comb[0].split('-')
    checkout_date = comb[1].split('-')
    checking_year = checking_date[0]
    checking_month = checking_date[1]
    checking_monthday =checking_date[2]
    checkout_year= checkout_date[0]
    checkout_month= checkout_date[1]
    checkout_monthday= checkout_date[2]
    url = f"https://www.expedia.com/Hotel-Search?adults={numOfAdults}&d1={comb[0]}&d2={comb[1]}&destination={city}&endDate={comb[1]}&latLong={city_latlong}&regionId={city_region_id}&rooms={numOfRooms}&semdtl=&sort=RECOMMENDED&startDate={comb[0]}&theme=&useRewards=false&userIntent="
    single_search_df  = single_search(url, driver , Snapshot_date, checking_date_type, checkout_date_type)
    expedia_df = pd.concat([expedia_df, single_search_df])
    print(single_search_df)
#driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-stid="show-more-results"]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00E56643]
	(No symbol) [0x00DEBE21]
	(No symbol) [0x00CEDA9D]
	(No symbol) [0x00D21342]
	(No symbol) [0x00D2147B]
	(No symbol) [0x00D58DC2]
	(No symbol) [0x00D3FDC4]
	(No symbol) [0x00D56B09]
	(No symbol) [0x00D3FB76]
	(No symbol) [0x00D149C1]
	(No symbol) [0x00D15E5D]
	GetHandleVerifier [0x010CA142+2497106]
	GetHandleVerifier [0x010F85D3+2686691]
	GetHandleVerifier [0x010FBB9C+2700460]
	GetHandleVerifier [0x00F03B10+635936]
	(No symbol) [0x00DF4A1F]
	(No symbol) [0x00DFA418]
	(No symbol) [0x00DFA505]
	(No symbol) [0x00E0508B]
	BaseThreadInitThunk [0x769200F9+25]
	RtlGetAppContainerNamedObjectPath [0x77377BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77377B8E+238]


In [10]:
expedia_df

,Snapshot,Index,Hotel Name,TTT,LOS,Grade,Num of Reviews,Curr Price,Original Price,Distance from center,Type of room,Location grade,Extra Data
0,2023-02-07 11:13:13.400517,1,Hotel 32 32,0,1,8.0,None,170,227,None,None,None,None
1,2023-02-07 11:13:13.400517,2,Omni Berkshire Place,0,1,9.0,None,269,299,None,None,None,None
2,2023-02-07 11:13:13.400517,3,New York Marriott Marquis,0,1,8.8,None,329,None,None,None,None,None
3,2023-02-07 11:13:13.400517,4,"Element New York Times Square West, a Marriott...",0,1,7.8,None,107,119,None,None,None,None
4,2023-02-07 11:13:13.400517,5,Truss Hotel Times Square,0,1,7.8,None,116,129,None,None,None,None
5,2023-02-07 11:13:13.400517,6,Park Central Hotel New York,0,1,8.0,None,98,109,None,None,None,None
6,2023-02-07 11:13:13.400517,7,Pod Times Square,0,1,8.6,None,89,None,None,None,None,None
7,2023-02-07 11:13:13.400517,8,Hyatt Place New York City/Times Square,0,1,7.8,None,119,None,None,None,None,None
8,2023-02-07 11:13:13.400517,9,Hilton Garden Inn Long Island City New York,0,1,8.6,None,149,None,None,None,None,None


In [59]:
len(expedia_df)

18

In [51]:
path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': r"C:\Users\yarin\chromedriver.exe", 'yair':'C:\Windows\chromdriver.exe' }
driver = webdriver.Chrome(path['yarin'])
city = "New%20York%20%28and%20vicinity%29%2C%20New%20York%2C%20United%20States%20of%20America"
city_latlong = 'latLong=40.75668%2C-73.98647'
city_region_id = '178293'
numOfAdults=2
numOfChild=0
numOfRooms=1
checking_year = 2023
checking_month = 3
checking_monthday = 5
checkout_year= 2023
checkout_month= 3
checkout_monthday= 8
url = f"https://www.expedia.com/Hotel-Search?adults={numOfAdults}&d1={comb[0]}&d2={comb[1]}&destination={city}&endDate={comb[1]}&latLong={city_latlong}&regionId={city_region_id}&rooms={numOfRooms}&semdtl=&sort=RECOMMENDED&startDate={comb[0]}&theme=&useRewards=false&userIntent="
driver.get(url)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


time.sleep(2) 
button=driver.find_element(by =By.CSS_SELECTOR,value =f'[data-stid="show-more-results"]')
button.click()
time.sleep(2) 


text_boxes_list = driver.find_elements(by= By.CLASS_NAME , value="uitk-card uitk-card-roundcorner-all uitk-card-has-primary-theme")

print(len(text_boxes_list))

0


In [41]:
t_box = text_boxes_list[0]
hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Grade":None,
                "Num of Reviews":None,"Curr Price":None,"Original Price":None,"Distance from center":None,
                "Type of room":None,"Location grade":None ,"Extra Data":None}
print(t_box)
try:
    
    
    name = t_box.find_element(by= By.CLASS_NAME , value="uitk-card-content-section uitk-card-content-section-padded uitk-layout-grid-item uitk-layout-grid-item-has-column-start-by-medium").text
    '''score = t_box.find_element(By.CLASS_NAME, 'uitk-spacing-padding-inlineend-half').text
    price = t_box.find_element(By.CSS_SELECTOR, "div[data-test-id='price-summary-message-line']").text.split('$')
    
    hotel['Snapshot'] = Snapshot_date
    hotel['Hotel Name'] = name
    hotel['TTT'] = (checking_date_type - Snapshot_date).days
    hotel['LOS'] = (checkout_date_type - checking_date_type).days
    hotel['Grade'] = score 
    hotel['Curr Price'] = price[-1]
    if len(price) > 3:
        hotel['Original Price'] = price[1].split('\n')[0]'''

except:
     print('error')
     
print(name)

<selenium.webdriver.remote.webelement.WebElement (session="3a18be3f4be38b9cad3a909c45939626", element="e0888c2c-702f-49a9-b9fd-5b7396cdf32c")>
error


NameError: name 'name' is not defined

In [101]:
path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': r"C:\Users\yarin\chromedriver.exe", 'yair':'C:\Windows\chromdriver.exe' }
driver = webdriver.Chrome(path['yarin'])
city = "New%20York%20%28and%20vicinity%29%2C%20New%20York%2C%20United%20States%20of%20America"
city_latlong = 'latLong=40.75668%2C-73.98647'
city_region_id = '178293'
numOfAdults=2
numOfChild=0
numOfRooms=1
checking_year = 2023
checking_month = 3
checking_monthday = 5
checkout_year= 2023
checkout_month= 3
checkout_monthday= 8
url = f"https://www.expedia.com/Hotel-Search?adults={numOfAdults}&d1={comb[0]}&d2={comb[1]}&destination={city}&endDate={comb[1]}&latLong={city_latlong}&regionId={city_region_id}&rooms={numOfRooms}&semdtl=&sort=RECOMMENDED&startDate={comb[0]}&theme=&useRewards=false&userIntent="
driver.get(url)



text_boxes_list = driver.find_elements(By.XPATH , "//div[@class = 'uitk-spacing uitk-spacing-margin-blockstart-three']")


print(len(text_boxes_list))

20


In [144]:
# extract values from card:

name  = t_box.find_element(By.XPATH, "//div[@class = 'uitk-spacing uitk-spacing-padding-blockend-three uitk-layout-flex-item']").text.split('\n')[0]
grade = t_box.find_element(By.XPATH, "//span[@class = 'uitk-spacing uitk-spacing-padding-inlineend-half uitk-layout-flex-item']").text
reviews = t_box.find_element(By.XPATH, '//*[@id="app-layer-base"]/div[1]/main/div/div/div/div/div[2]/section[2]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/div/div[2]/div/div[2]/div[1]/div/div/div/div/span[4]/span').text.split('(')[1].split(' ')[0]
curr_price = t_box.find_element(By.XPATH, '//*[@id="app-layer-base"]/div[1]/main/div/div/div/div/div[2]/section[2]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[1]/div/div').text.split(' ')[-1]
original_price = t_box.find_element(By.XPATH, "//div[@class = 'uitk-text uitk-type-300 uitk-text-default-theme is-visually-hidden']").text.split(' ')[-1]

In [127]:
driver.find_elements(By.XPATH , "//div[@class = 'uitk-spacing uitk-spacing-padding-inlineend-one uitk-layout-flex-item']")

[]

In [ ]:
//*[@id="app-layer-base"]/div[1]/main/div/div/div/div/div[2]/section[2]/div/div[2]/div/div[2]/div[1]/div[1]/div[2]

In [ ]:
//*[@id="app-layer-base"]/div[1]/main/div/div/div/div/div[2]/section[2]/div/div[2]/div/div[2]/div[1]/div[1]/div[4]